In [ ]:
import numpy as np 
import pandas as pd 

import os
print(os.listdir("../input"))

**Objective** 

The objective of this kernel is to explore the House Prices dataset and generate a clean dataset to train the models. Feel free to use the resulting clean datasets for modelling, but add a reference pointing to this kernel.


**Load dataset**

First of all let's read the training and test datasets.

In [ ]:
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

**Preprocessing**

As indicated in the [documentation](http://ww2.amstat.org/publications/jse/v19n3/Decock/DataDocumentation.txt), this dataset has some outliers. First we will explore and remove those and then we will inspect and clean the NULL values in the dataset for each variable. 

In [ ]:
train.plot.scatter('GrLivArea', 'SalePrice')

Gotcha! We can see there are indeed two points with GrLivArea > 4000 and SalePrice around 150-200k. Such a bargain! Bear in mind the variable GrLivArea specifies the living area in square feet, so it is not surprising to observe a somewhat linear relation between living area and house price. We will remove points with GrLivArea > 4000 and SalePrice < 300000..

In [ ]:
train = train.drop(train[(train['GrLivArea']>4000) & (train['SalePrice']<300000)].index)

Okay, time to clean NULL values. First let's explore how many NULL values we have per attribute.

In [ ]:
null_values_train = train.isnull().sum()
print(null_values_train[null_values_train>0].sort_values(ascending=False))

There we go, we have 19 attributes with missing values ranging from above 99% missing values to around 7%. Let's see if the missing values in test set are the same.

In [ ]:
null_values_test = test.isnull().sum()
print(null_values_test[null_values_test>0].sort_values(ascending=False))

Okay, there are some other fields missing in test set. We have to be careful during imputation so as not to leak data from test. Let's explore and address them one by one.

** Cleaning Missing values **

- PoolQC: indicates Pool quality. According to the documentation above, a missing value indicates there is no pool. We will just substitute NULL values for "None".
- MiscFeature: indicates other features not covered in other fields (pools, tennis courts...). A missing values indicates there is no such feature. We will substitute NULL values for "None".
- Alley: type of alley access to property. A values of NULL indicates no alley access. We will substitue NULL values for "None".
- Fence: fence quality. A NULL values indicates no Fence, therefore we will substitute NULL values for "None".
- FireplaceQu: fireplace quality. A NULL value indicates no fireplace. We will replace NULL for "None".
- LotFrontage: linear feet of street connected to property. We will assume that it will be similar to neighbours, therefore we will substitute NULL values by the median of neighbouring houses grouped by "Neighborhood".
- GarageYrBlt: year garage was built. We will substitute NULL values by 0 (no garage).
- GarageType: garage location. We will replace NULL values by "None".
- GarageFinish: interior finish of the garage. Will substitute NULL values by "None".
- GarageQual: garage quality. We will substitute NULL values by 0 (no garage).
- GarageCond: garage condition. Will substitute NULL values by "None".
- BsmtFinType2: rating of basement finished area (if multiple types). Will substitute NULL values by "None".
- BsmtExposure: walkout or garden level walls. Will substitute NULL values by "None".
- BsmtFinType1: rating of basement finished area. Will substitute NULL values by "None".
- BsmtCond: general condition of the basement. Will substitute NULL values by "None".
- BsmtQual: height of the basement as ordinal variable. Will substitute NULL values by "None".
- MasVnrArea: masonry veneer area in square feet. We will substitute NULL values by 0 (no masonry).
- MasVnrType: masonry veneer type. Will substitute NULL values by "None".
- Electrical: electrical system. Just one NULL value, we will substitute by most common value.
- MSZoning: general zoning classification of the sale. Since there are a few NULL values only we will substitute by most common value.
- Functional: home functionality. Documentation indicates default is typical, hence we substitute NULL by "Typ".
- BsmtHalfBath: basement half bathrooms. We will substitute NULL values by 0 (no bathroom).
- BsmtFullBath: basement full bathrooms. We will substitute NULL values by 0 (no bathroom).
- Utilities: type of utilities available. Since there are a few NULL values only we will substitute by most common value.
- SaleType: type of sale. Since there are a few NULL values only we will substitute by most common value.
- GarageArea: size of garage in square feet. We will substitute NULL values by 0 (no garage).
- GarageCars: size of garage in car capacity. We will substitute NULL values by 0 (no garage).
- KitchenQual: kitchen quality.  Since there are a few NULL values only we will substitute by most common value.
- TotalBsmtSF: total square feet of basement area. We will substitute NULL values by 0 (no basement).
- BsmtUnfSF: unfinished square feet of basement area. We will substitute NULL values by 0 (no basement).
- BsmtFinSF2: type 2 finished square feet. We will substitute NULL values by 0 (no basement).
- BsmtFinSF1: type 1 finished square feet. We will substitute NULL values by 0 (no basement).
- Exterior2nd: exterior covering on house (if more than one material). Since there are a few NULL values only we will substitute by most common value.
- Exterior1st: exterior covering on house. Since there are a few NULL values only we will substitute by most common value.

Since most variables will be substituted by the same values, we will do it in several steps: one for categorical variables that we want to substitute NULL by "None", one by numerical variables that we want to substitute NULL by 0, one for variables with few NULL values that we want to substitute by the most common value and we will treat the LotFrontage and Functional variables on their own.


In [ ]:
def cleanNullValues(df,train_df):
    """
    Clean the NULL values from the dataframe df.
    
    - To impute LotFrontage we create a dictionary with Neighborhood->median LotFrontage 
    using the training set.
    - To impute attributes by most common value we use training set most common value.
    
    This prevents data leakage from the test set during imputation.
    """
    #Categorical variables
    categorical = ['PoolQC','MiscFeature','Alley','Fence','FireplaceQu','GarageType','GarageFinish',
                   'GarageCond','BsmtFinType2','BsmtExposure','BsmtFinType1','BsmtCond','BsmtQual',
                   'MasVnrType']
    for col in categorical:
        df[col] = df[col].fillna('None')

    #Numerical variables
    numerical = ['GarageYrBlt','GarageQual','MasVnrArea','BsmtHalfBath','BsmtFullBath','GarageArea',
                 'GarageCars','TotalBsmtSF','BsmtUnfSF','BsmtFinSF2','BsmtFinSF1']
    for col in numerical:
        df[col] = df[col].fillna(0)
        
    #Few NULL values
    few = ['Electrical','MSZoning','Utilities','SaleType','KitchenQual','Exterior2nd','Exterior1st']
    for col in few:
        df[col] = df[col].fillna(train[col].mode()[0])
    
    #Functional
    df['Functional'] = df['Functional'].fillna('Typ')

    #LotFrontage (Adapted from https://www.kaggle.com/serigne/stacked-regressions-top-4-on-leaderboard)
    neighborhood2lotfrontage = train.loc[:,['Neighborhood','LotFrontage']].groupby(
                                       ['Neighborhood']).median()['LotFrontage'].to_dict()
    for key, _ in df.groupby("Neighborhood")["LotFrontage"]:
        df["LotFrontage"] = df.groupby("Neighborhood")["LotFrontage"].transform(
            lambda x: x.fillna(neighborhood2lotfrontage[key]))
        
    return df


In [ ]:
train = cleanNullValues(train,train)
test = cleanNullValues(test,train)

#Check that all NULL values are cleaned
print('Training set:')
print(sum(train.isnull().sum()>0))
print('Test set:')
print(sum(test.isnull().sum()>0))

** Write clean files **

Finally we write the clean files for our models to use!

In [ ]:
train.to_csv('train_clean.csv',index=False)
test.to_csv('test_clean.csv',index=False)

** Final notes **

- Check this [kernel](https://www.kaggle.com/gunbl4d3/bayesian-optimization-for-house-prices/output) for an example on how to use this dataset to produce a submission for the [House Prices: Advanced Regression Techniques competition](https://www.kaggle.com/c/house-prices-advanced-regression-techniques).